## import libraries

In [1]:
import torch
import torch.utils.data as data
import torchvision.transforms.v2 as v2
import torchvision
import torch.nn as nn
import torch.optim as optim

## Sequential

In [2]:
model = nn.Sequential(
    nn.Linear(28*28, 32),
    nn.ReLU(),
    nn.Linear(32, 10)
)

In [3]:
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=10, bias=True)
)


In [4]:
model = nn.Sequential()
model.add_module('layer 1', nn.Linear(28*28, 32))
model.add_module('ReLU 1', nn.ReLU())

In [5]:
print(model)

Sequential(
  (layer 1): Linear(in_features=784, out_features=32, bias=True)
  (ReLU 1): ReLU()
)


In [6]:
block = nn.Sequential(
    nn.Linear(32, 32),
    nn.LeakyReLU(),
    nn.Linear(32, 10),
    nn.LeakyReLU()
)

In [7]:
model.add_module('block', block)
model

Sequential(
  (layer 1): Linear(in_features=784, out_features=32, bias=True)
  (ReLU 1): ReLU()
  (block): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=32, out_features=10, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
)

## ModelList

### problem

In [8]:
class ModelNN(nn.Module):
  def __init__(self,input_dim, output_dim, n_layer=3):
    super().__init__()
    self.layers = [nn.Linear(input_dim//n, input_dim//(n+1)) for n in range(1, n_layer+1)]

    sz_input = self.layers[-1].out_features
    self.layer_out = nn.Linear(sz_input, output_dim)

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
      x = nn.functional.tanh(x)

    x = self.layer_out(x)
    return x

In [9]:
model = ModelNN(28*28, 10)

In [10]:
model

ModelNN(
  (layer_out): Linear(in_features=196, out_features=10, bias=True)
)

In [11]:
list(model.parameters()) # !

[Parameter containing:
 tensor([[ 0.0677, -0.0036,  0.0183,  ..., -0.0072, -0.0323, -0.0365],
         [-0.0156, -0.0366, -0.0242,  ...,  0.0448, -0.0553,  0.0044],
         [-0.0412, -0.0241,  0.0242,  ..., -0.0583, -0.0681, -0.0250],
         ...,
         [-0.0065,  0.0295,  0.0674,  ..., -0.0065, -0.0393,  0.0314],
         [ 0.0254, -0.0193, -0.0565,  ...,  0.0407, -0.0267, -0.0668],
         [-0.0672, -0.0547, -0.0223,  ...,  0.0259, -0.0029, -0.0706]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0101, -0.0212, -0.0221, -0.0480, -0.0078,  0.0377, -0.0520,  0.0093,
         -0.0082, -0.0230], requires_grad=True)]

### solution

In [12]:
class ModelNN(nn.Module):
  def __init__(self,input_dim, output_dim, n_layer=3):
    super().__init__()
    self.layers = nn.ModuleList(
        [nn.Linear(input_dim//n, input_dim//(n+1)) for n in range(1, n_layer+1)]
    )
    #self.layers = [nn.Linear(input_dim//n, input_dim//(n+1)) for n in range(1, n_layer+1)]
    #self.layers = nn.Sequential(self.layers)

    sz_input = self.layers[-1].out_features
    self.layer_out = nn.Linear(sz_input, output_dim)

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
      x = nn.functional.tanh(x)

    x = self.layer_out(x)
    return x

In [13]:
model = ModelNN(28*28, 10)

In [14]:
model

ModelNN(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=392, bias=True)
    (1): Linear(in_features=392, out_features=261, bias=True)
    (2): Linear(in_features=261, out_features=196, bias=True)
  )
  (layer_out): Linear(in_features=196, out_features=10, bias=True)
)

In [15]:
len(list(model.parameters()))

8

### solution 2

In [16]:
class ModelNN(nn.Module):
  def __init__(self,input_dim, output_dim, n_layer=3):
    super().__init__()
    self.layers = nn.ModuleList()

    for n in range(1, n_layer+1):
      self.layers.add_module(f'layer_{n}', nn.Linear(input_dim//n, input_dim//(n+1)))

    self.layer_out = nn.Linear(input_dim//(n_layer+1), output_dim)

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
      x = nn.functional.tanh(x)

    x = self.layer_out(x)
    return x

In [17]:
model = ModelNN(28*28, 10)

In [18]:
model

ModelNN(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=392, bias=True)
    (1): Linear(in_features=392, out_features=261, bias=True)
    (2): Linear(in_features=261, out_features=196, bias=True)
  )
  (layer_out): Linear(in_features=196, out_features=10, bias=True)
)

In [19]:
model.layers.layer_1

Linear(in_features=784, out_features=392, bias=True)

## ModelDict

In [20]:
class ModelNN(nn.Module):
  def __init__(self,input_dim, output_dim, n_layer=3, act_type=None):
    super().__init__()
    self.layers = nn.ModuleList()
    self.act_type = act_type

    for n in range(1, n_layer+1):
      self.layers.add_module(f'layer_{n}', nn.Linear(input_dim//n, input_dim//(n+1)))

    self.layer_out = nn.Linear(input_dim//(n_layer+1), output_dim)
    self.act_lst = nn.ModuleDict({
        'relu': nn.ReLU(),
        'lk_relu': nn.LeakyReLU()
    })

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
      if self.act_type and self.act_type in self.act_lst:
        x = self.act_lst[self.act_type](x)

    x = self.layer_out(x)
    return x

In [21]:
model = ModelNN(28*28, 10, act_type='relu')

In [22]:
model

ModelNN(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=392, bias=True)
    (1): Linear(in_features=392, out_features=261, bias=True)
    (2): Linear(in_features=261, out_features=196, bias=True)
  )
  (layer_out): Linear(in_features=196, out_features=10, bias=True)
  (act_lst): ModuleDict(
    (relu): ReLU()
    (lk_relu): LeakyReLU(negative_slope=0.01)
  )
)